In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

2024-03-21 12:06:29.138802: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-21 12:06:29.209427: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-21 12:06:30.298678: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
all_players = ['Бот','Вадим', 'Ваня', 'Гриша', 'Данил', 'Ден', 'Джун', 'Миша', 'Ондрей', 'Савва', 'Сеньор', 'Юран']
input_NN_line = [player+"_team1" for player in all_players] + [player+"_team2" for player in all_players]
shown_players =all_players[1:]
# make the list of players from table above
old_method_dataset = pd.read_csv("old_method_dataset.csv")
print (old_method_dataset.head(15))

    Player  Games  Wins  Winrate
0   Сеньор     21    14     0.67
1    Гриша     22    14     0.64
2     Миша     19    11     0.58
3     Ваня     18    10     0.56
4      Бот      6     3     0.50
5     Джун      8     4     0.50
6    Данил     16     8     0.50
7    Вадим     24    10     0.42
8   Ондрей     12     5     0.42
9     Юран     10     4     0.40
10     Ден     18     6     0.33
11   Савва     10     3     0.30


In [3]:
def add_data(names_win, names_lose, campaign='Map', date = ''):
    old_method_dataset = pd.read_csv("old_method_dataset.csv")
    if len(names_win)>len(names_lose):
        names_lose += ['Бот']
    elif len(names_win)<len(names_lose):
        names_win += ['Бот']

    for name in names_win:
        old_method_dataset.loc[old_method_dataset['Player'] == name, 'Wins'] += 1
        old_method_dataset.loc[old_method_dataset['Player'] == name, 'Games'] += 1
    for name in names_lose:
        old_method_dataset.loc[old_method_dataset['Player'] == name, 'Games'] += 1
    old_method_dataset["Winrate"] = old_method_dataset["Wins"] / old_method_dataset["Games"]
    old_method_dataset = old_method_dataset.sort_values(by="Winrate", ascending=False)
    old_method_dataset["Winrate"]=old_method_dataset["Winrate"].apply(lambda x: round(x, 2))

    old_method_dataset.to_csv("old_method_dataset.csv", index=False)
    print(old_method_dataset.to_markdown()) 

    # columns - Date,Map,Player1,Player2,Player3,Player4,Player5,Player6,Player7,Player8,TeamWon
    # add to datasetNN a line with date, map, team1, team2, whichteamwon:
    # e.g. 2024-03-09,Map,Ден,Гриша,Ондрей,Юран,Сеньор,Савва,Данил,Бот,1.0

    while len(names_win)< 4:
        names_win += ['Бот']
    while len(names_lose)< 4:
        names_lose += ['Бот']
    team1 = ','.join(names_win)
    team2 = ','.join(names_lose)
    new_line = f'{date},{campaign},{team1},{team2},1\n'
    with open('datasetNN.csv', 'a') as file:
        file.write(new_line)
    print('Added to datasetNN.csv: ', new_line) 
    return True

In [4]:
def get_teams(present_players):
    if len(present_players)>8: 
        return "Too many players"
    
    all_teams_old_method = pd.DataFrame(columns=["Team1", "Team1 Winrate", "Team2", "Team2 Winrate"])
    all_teams_nn = pd.DataFrame(columns=["Team1", "Team2","NN output"])
    input_nn =pd.DataFrame(columns=input_NN_line)

    if len(present_players)%2==1:
        present_players = present_players + ["Бот"]  

    for i in range(0, len(present_players)):
        for j in range(i+1, len(present_players)):
            for k in range(j+1, len(present_players)):
                if len(present_players)==6:
                    team1 = [present_players[i], present_players[j], present_players[k]]
                    team2 = [x for x in present_players if x not in team1]
                    team1_winrate = old_method_dataset[old_method_dataset["Player"].isin(team1)]["Winrate"].sum()
                    team2_winrate = old_method_dataset[old_method_dataset["Player"].isin(team2)]["Winrate"].sum()
                    all_teams_old_method.loc[len(all_teams_old_method)] = [team1, team1_winrate, team2, team2_winrate]
                    # create a row with the players
                    row = np.zeros(len(input_NN_line))
                    for player in team1:
                        row[input_NN_line.index(player+"_team1")] = 1
                    for player in team2:
                        row[input_NN_line.index(player+"_team2")] = 1
                    # add row to teams_nn
                    input_nn.loc[len(input_nn)] = row
                    all_teams_nn.loc[len(all_teams_nn)] = [team1, team2, 0]
                if len(present_players)==8:
                    for l in range(k+1, len(present_players)):
                        team1 = [present_players[i], present_players[j], present_players[k], present_players[l]]
                        team2 = [x for x in present_players if x not in team1]
                        team1_winrate = old_method_dataset[old_method_dataset["Player"].isin(team1)]["Winrate"].sum()
                        team2_winrate = old_method_dataset[old_method_dataset["Player"].isin(team2)]["Winrate"].sum()
                        all_teams_old_method.loc[len(all_teams_old_method)] = [team1, team1_winrate, team2, team2_winrate]
                        # create a row with the players
                        row = np.zeros(len(input_NN_line))
                        for player in team1:
                            row[input_NN_line.index(player+"_team1")] = 1
                        for player in team2:
                            row[input_NN_line.index(player+"_team2")] = 1
                        # add row to teams_nn
                        input_nn.loc[len(input_nn)] = row
                        all_teams_nn.loc[len(all_teams_nn)] = [team1, team2, 0]
              

    all_teams_old_method.sort_values(by="Team1 Winrate", inplace=True)
    all_teams_old_method.reset_index(drop=True, inplace=True)

    # predict the output of the NN
    model = keras.models.load_model('model.keras')
    all_teams_nn["NN output"] = model.predict(input_nn)
    all_teams_nn.sort_values(by="NN output", inplace=True)

    return all_teams_old_method, all_teams_nn

In [5]:
present_players = ['Вадим', 'Ваня', 'Гриша', 'Ден', 'Савва', 'Сеньор', 'Юран']

In [39]:
results_old, results_nn = get_teams (present_players)
middle_nn = len(results_nn)//2
middle_old = len(results_old)//2

# find closest result to 0.5 in NN
closest = results_nn.iloc[(results_nn["NN output"]-0.5).abs().argsort()[:1]]
#  find the middle entry in the nn
median = results_nn.iloc [[middle_nn]]
median2 = results_nn.iloc [[middle_nn+1]]

# concat previous dataframes
nn_output = pd.concat([closest, median, median2])
# set indeces of nn_output to "Best","Median","Median2"
nn_output.index = ["Best","Median","Median2"]



# find closest result to 0.5 in old method
closest_old = results_old.iloc[[middle_old]]
closest2_old = results_old.iloc[[middle_old+1]]

old_method_output = pd.concat([closest_old, closest2_old])
old_method_output.index = ["Best","Second Best"]



3/3 [==============================] - 0s 5ms/step
